In [1]:
using Laplacians

objc[18709]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18709]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18709]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18709]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_2/"

"../graphs/se_2/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [9]:
@time F = treeSolver(tree)
@time F(b);

  0.429205 seconds (281.03 k allocations: 11.707 MB, 2.01% gc time)
  0.205105 seconds (118.54 k allocations: 5.064 MB)


In [10]:
# a run on standard Float64 data types
numIts = 1000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

  1.000697 seconds (966.30 k allocations: 38.698 MB, 1.60% gc time)


In [11]:
numIts = 1000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 7.49564838471667
Working on iteration 20 with a-norm error 1.833961910450398e-5
Working on iteration 30 with a-norm error 1.9972988741371817e-9
Working on iteration 40 with a-norm error 1.997291945867776e-9
Working on iteration 50 with a-norm error 1.9972184896757584e-9
Working on iteration 60 with a-norm error 1.9972168720828467e-9
Working on iteration 70 with a-norm error 1.9972167978926867e-9
Working on iteration 80 with a-norm error 1.997216793153102e-9
Working on iteration 90 with a-norm error 1.9972167935909828e-9
Working on iteration 100 with a-norm error 1.997216793872611e-9
Working on iteration 110 with a-norm error 1.9972167941559914e-9
Working on iteration 120 with a-norm error 1.9972167948051255e-9
Working on iteration 130 with a-norm error 1.9972167962070527e-9
Working on iteration 140 with a-norm error 1.99721679821978e-9
Working on iteration 150 with a-norm error 1.9972168156918094e-9
Working on iteration 160 with a-norm error 1.

In [12]:
?pcgV

search: pcgV pcgSolver pcgLapSolver pcg pcgWorker ProcessGroup productGraph



No documentation found.

`pcgV` is a generic `Function`.

```julia
# 1 method for generic function "pcgV":
pcgV{Tv}(mat, b::Array{Tv,1}, pre::Function, lhs::Array{Tv,1}) at /Users/serbanstan/git/TreePCG/julia/verbosePCG.jl:2
```
